In [1]:
import math

In [2]:
import math
class Value:
  def __init__(self, data, children = (), op=''):
    self.data = data
    self._prev = set(children)
    self._op = op
    self.grad = 0.0
    self._backward = lambda:None

  def __repr__(self):
    return f'Value(data={self.data})'

  def __add__(self,other):
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data+other.data,(self,other), '+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __radd__(self,other):
    return self+other

  def __sub__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data+other.data,(self,other), '-')
    def _backward(self):
      self.grad += 1.0 * out.grad
      other.grad += -1.0 * out.grad
    return out

  def __rmul__(self,other):
    return self*other

  def __mul__(self, other):
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data*other.data, (self,other), '*')

    def _backward():
      self.grad += other.data
      other.grad += self.data
    out._backward = _backward
    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
    out = Value(t, (self,), 'tanh')

    def _backward():
      self.grad += (1-(t**2)) * out.grad

    out._backward = _backward
    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    for node in reversed(topo):
      node._backward()
